# Energy Calculation

In [36]:
import pandas as pd
import numpy as np
import geopandas as gpd


In [37]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)

In [38]:
# Read DataFrame
pkls_path = pkls = r'D:\Tesis\ResEleCon-MX\pickles'
enigh = pd.read_pickle(pkls_path + '\concentrador_gasfix_2018.pkl')

## Tariff from CFE database

In [39]:
# Read cfe totals per municipality and tariff
cols = list(range(0,6))+[13]
CFE_mun = pd.read_csv("D:/Tesis/Datos/Cosumo por sectores por año/consumodeelectricidadpormunicipio.csv", usecols=cols, header=2)

CFE_mun['2017'] = (CFE_mun['2017'].replace({',': ''}, regex=True)).astype(np.int64)

# Assign corresponding ZM to each municipality
CFE_mun['CVE_MUN'] = (CFE_mun['Cve Inegi']*1000 + CFE_mun['Cve Mun'])
#CFE_mun = CFE_mun.merge(ZM_2015[['CVE_MUN','CVE_ZM']], on='CVE_MUN')

# Filter Residential consumption
domestic = ['1','1A','1B','1C','1D','1E','1F','1F','DAC']
CFE_mun['Tarifa'] = CFE_mun['Tarifa'].str.strip()
CFE_res = CFE_mun.loc[CFE_mun['Tarifa'].isin(domestic)]

# Select tariff with highest consumption
tarifa_cfe = CFE_res[['CVE_MUN','Tarifa']].loc[CFE_res.groupby('CVE_MUN')['2017'].idxmax()]
enigh_tar = enigh.merge(tarifa_cfe, on='CVE_MUN', how='left')

In [40]:
enigh_tar.head()

,folioviv,foliohog,CVE_MUN,estado,publico,CVE_ZM,NOM_ZM,CVE_ENT,NOM_ENT,NOM_MUN,tam_loc,est_socio,est_dis,upm,factor,clase_hog,sexo_jefe,edad_jefe,educa_jefe,tot_integ,hombres,mujeres,mayores,menores,ocupados,ing_cor,gasto_mon,energia,vehiculos,tipo_viv,antiguedad,num_cuarto,disp_elect,focos_inca,focos_ahor,combustible,tenencia,calent_sol,calent_gas,tanque_gas,aire_acond,calefacc,gasto_tri_ele,gasto_tri_gas,gasto_tri_lpg,gasto_tri_oil,gasto_tri_diesel,gasto_tri_coal,gasto_tri_wood,gasto_tri_heat,gasto_tri_Magna,gasto_tri_Premium,gasto_tri_Die-Gas,gasto_tri_total_gas,gasto_tri_total_gasolina,Tarifa
0,100013601,1,1001,1,0.00,1.01,Aguascalientes,1,Aguascalientes,Aguascalientes,1,3,2,1,175,2,1,74,4,3,2,1,3,0,2,76403.70,18551.47,3600.0,1,1.0,42.0,6,1,0.0,6.0,4.0,4,1,1,2,2,2,600.0,3000.0,NaN,NaN,NaN,NaN,NaN,NaN,7200.0,NaN,NaN,3000.0,7200.0,1
1,100013602,1,1001,1,578.56,1.01,Aguascalientes,1,Aguascalientes,Aguascalientes,1,3,2,1,175,2,1,48,11,5,2,3,5,0,2,42987.73,55470.99,495.0,2,1.0,40.0,6,1,0.0,12.0,4.0,2,2,1,2,2,2,450.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3600.0,NaN,NaN,NaN,3600.0,1
2,100013603,1,1001,1,4255.68,1.01,Aguascalientes,1,Aguascalientes,Aguascalientes,1,3,2,1,175,2,1,39,10,2,1,1,2,0,2,580697.74,103106.89,2475.0,1,1.0,50.0,7,1,0.0,12.0,3.0,4,2,1,1,2,2,375.0,NaN,2100.0,NaN,NaN,NaN,NaN,NaN,NaN,4500.0,NaN,2100.0,4500.0,1
3,100013604,1,1001,1,0.00,1.01,Aguascalientes,1,Aguascalientes,Aguascalientes,1,3,2,1,175,2,2,70,8,2,1,1,2,0,0,46252.71,19340.06,858.0,1,1.0,45.0,5,1,0.0,7.0,4.0,4,1,1,2,2,2,480.0,378.0,NaN,NaN,NaN,NaN,NaN,NaN,6000.0,NaN,NaN,378.0,6000.0,1
4,100013606,1,1001,1,0.00,1.01,Aguascalientes,1,Aguascalientes,Aguascalientes,1,3,2,1,175,2,2,51,4,4,1,3,3,1,2,53837.09,13605.03,300.0,0,1.0,40.0,4,1,0.0,7.0,4.0,2,2,1,2,2,2,300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1


## Tarifas CFE

### DAC Regions
Fot domestic high consumption (DAC) the applied tariff depends on the a different spatial distribution. The regions are: Baja California, Baja California Sur, Centro, Noroeste, Norte-Noreste y Sur-Peninsular.
https://energypedia.info/images/f/fb/GIZ_Tutorial_Tarifas_El%C3%A9ctricas_2015.pdf

In [41]:
# Region
list_edomex = [15108,15109,15039,15025,15037,15076,15106,15012,15024,15020,15121,15013,15104,15057,15033,15031,15029,15099,17007]
list_NE = [30123,30133,24011,24024]

conditions = [
    (enigh_tar['estado']==2),
    (enigh_tar['estado']==3),
    (enigh_tar['estado']==9) | (enigh_tar['CVE_MUN'].isin(list_edomex)),
    (enigh_tar['estado'].isin([25,26])),
    (enigh_tar['estado'].isin([5,8,10,19,28])) | (enigh_tar['CVE_MUN'].isin(list_NE)),
    (enigh_tar['estado'].isin([1,4,6,7,11,12,13,14,15,16,17,18,20,21,22,23,24,27,29,30,31,32])) & (~enigh_tar['CVE_MUN'].isin(list_NE+list_edomex))]
regiones = ['BC','BCS','Ce','NO','NNE','SP']
enigh_tar['Region'] = np.select(conditions, regiones, default=np.nan)

### Tariff
The prices taken are those of 2018 and the months from June to Octuber. This because the survey was carried out between August 21th and November 28th in 2018. As there are no information of the exact dates for the surveys, the values of the different months were averaged, so were the values of summer and non-summer months.

- 1-1F: https://app.cfe.mx/Aplicaciones/CCFE/Tarifas/Tarifas/tarifas_casa.asp
- DAC:  https://app.cfe.mx/Aplicaciones/CCFE/Tarifas/Tarifas/tarifas_casa.asp?Tarifa=DAC2003&Anio=2016&mes=7&imprime=

In [42]:
tarifas = ['1','1A','1B','1C','1D','1E','1F']

# Create df for tariffs
cfe_dic = {'tar': tarifas, 
           'Basico': [0.793,0.793,0.793,0.793,0.793,0.793,0.793], 
           'BasicoV': [np.nan,0.697,0.697,0.697,0.697,0.583,0.583,],
           'Intermedio': [0.956,0.956,0.956,0.956,0.956,0.956,0.956], 
           'IntermedioV': [np.nan,0.822,0.822,0.822,0.822,0.726,0.726],
           'IntermedioVA': [np.nan,np.nan,np.nan,1.05,1.05,0.948,1.768], 
           'Excedente': [2.802,2.802,2.802,2.802,2.802,2.802,2.802],
           'LimB': [75,75,75,75,75,75,75], 
           'LimBV': [np.nan,100,125,150,175,300,300], 
           'LimI': [140,150,175,175,200,200,200],
           'LimIV': [np.nan,150,225,300,400,750,1200], 
           'LimIVA': [np.nan,np.nan,np.nan,450,600,900,2500], 
           'Ldac': [250,300,400,850,1000,2000,2500]
          }

cfe = pd.DataFrame(data=cfe_dic)

cfe['LB_pesos'] = cfe['Basico']*cfe['LimB']
cfe['LBV_pesos'] = cfe['BasicoV']*cfe['LimBV']
cfe['LI_pesos'] = (cfe['LimI']-cfe['LimB'])*cfe['Intermedio']+cfe['LB_pesos']
cfe['LIV_pesos'] = (cfe['LimIV']-cfe['LimBV'])*cfe['IntermedioV']+cfe['LBV_pesos']
cfe['LIVA_pesos'] = (cfe['LimIVA']-cfe['LimIV'])*cfe['IntermedioVA']+cfe['LIV_pesos']
cfe['Ldac_pesos'] = (cfe['Ldac']-cfe['LimI'])*cfe['Excedente']+cfe['LI_pesos']
cfe.loc[0:3,'LdacV_pesos'] = (cfe['Ldac']-cfe['LimIV'])*cfe['Excedente']+cfe['LIV_pesos']
cfe.loc[3:,'LdacV_pesos'] = (cfe['Ldac']-cfe['LimIVA'])*cfe['Excedente']+cfe['LIVA_pesos']

cfe.set_index('tar', inplace=True)

# Create df for regions
dac_dic = {'Region': ['BC','BCS','Ce','NO','NNE','SP'], 
           'c_fijo_1': [100.63]*6, 'c_fijo_2': [100.77]*6, 'c_fijo_3': [102.24]*6, 'c_fijo_4': [102.45]*6, 'c_fijo_5': [102.79]*6, 'c_fijo_6': [102.80]*6,
           'c_fijo_7': [104.77]*6, 'c_fijo_8': [106.68]*6, 'c_fijo_9': [105.62]*6, 'c_fijo_10': [105.17]*6, 'c_fijo_11': [105.64]*6, 'c_fijo_12': [105.82]*6,
           'c_ener_1': [3.75,3.75,4.49,4.20,4.10,4.16], 'c_ener_2': [3.81,3.81,4.56,4.27,4.16,4.23], 'c_ener_3': [3.96,3.96,4.74,4.44,4.33,4.40], 'c_ener_4': [3.79,3.79,4.54,4.25,4.15,4.21], 'c_ener_5': [3.79,3.79,4.54,4.25,4.14,4.21], 'c_ener_6': [3.99,3.99,4.78,4.47,4.36,4.43],
           'c_ener_7': [4.08,4.08,4.89,4.58,4.46,4.53], 'c_ener_8': [4.11,4.11,4.92,4.61,4.49,4.56], 'c_ener_9': [3.97,3.97,4.75,4.45,4.34,4.41], 'c_ener_10': [4.01,4.01,4.81,4.50,4.39,4.46], 'c_ener_11': [4.13,4.13,4.95,4.64,4.52,4.59], 'c_ener_12': [4.21,4.21,5.04,4.72,4.60,4.67],
           
           'c_ener_v_1': [4.36,4.75,np.nan,np.nan,np.nan,np.nan], 'c_ener_v_2': [4.43,4.83,np.nan,np.nan,np.nan,np.nan], 'c_ener_v_3': [4.61,5.02,np.nan,np.nan,np.nan,np.nan], 'c_ener_v_4': [4.41,4.81,np.nan,np.nan,np.nan,np.nan], 'c_ener_v_5': [4.41,4.81,np.nan,np.nan,np.nan,np.nan], 'c_ener_v_6': [4.64,5.06,np.nan,np.nan,np.nan,np.nan],
           'c_ener_v_7': [4.75,5.18,np.nan,np.nan,np.nan,np.nan], 'c_ener_v_8': [4.78,5.21,np.nan,np.nan,np.nan,np.nan], 'c_ener_v_9': [4.62,5.03,np.nan,np.nan,np.nan,np.nan], 'c_ener_v_10': [4.67,5.09,np.nan,np.nan,np.nan,np.nan], 'c_ener_v_11': [4.81,5.25,np.nan,np.nan,np.nan,np.nan], 'c_ener_v_12': [4.90,5.34,np.nan,np.nan,np.nan,np.nan]
            }
dac = pd.DataFrame(data=dac_dic)

dac['c_fijo_v_m'] = dac.loc[:,'c_fijo_5':'c_fijo_10'].mean(axis=1)
dac['c_fijo_m'] = dac[['c_fijo_1','c_fijo_2','c_fijo_3','c_fijo_4','c_fijo_11','c_fijo_12']].mean(axis=1)
dac['c_ener_v_m'] = dac.loc[:,'c_ener_v_5':'c_ener_v_10'].mean(axis=1)
dac['c_ener_m'] = dac[['c_ener_1','c_ener_2','c_ener_3','c_ener_4','c_ener_11','c_ener_12']].mean(axis=1)
dac['c_ener_m_sm'] = dac.loc[:,'c_ener_5':'c_ener_10'].mean(axis=1) # Non-summer special tariff for summer months
dac.loc[0:1,'c_ener_m_sm'] = dac.loc[0:1,'c_ener_v_m']
dac.set_index('Region', inplace=True)

## MXN to kWh

In [43]:
tol_dac = 1 # tolerance for DAC limit

# Convert pesos to kWh
def mxn_to_kwh(gasto, tarifa, region, tipo_tar = False):
    gasto = gasto/1.16
    
    #not summer
    if gasto <= cfe.loc[tarifa,'LB_pesos']:
        kwh_nv = gasto/cfe.loc[tarifa,'Basico']
    elif gasto <= cfe.loc[tarifa,'LI_pesos']:
        kwh_nv = (gasto - cfe.loc[tarifa,'LB_pesos'])/cfe.loc[tarifa,'Intermedio'] + cfe.loc[tarifa,'LimB']
    #elif gasto <= cfe.loc[tarifa,'Ldac_pesos']*tol_dac:
    elif gasto <= (cfe.loc[tarifa,'Ldac']*dac.loc[region,'c_ener_m']+dac.loc[region,'c_fijo_m'])*tol_dac:
        kwh_nv = (gasto - cfe.loc[tarifa,'LI_pesos'])/cfe.loc[tarifa,'Excedente'] + cfe.loc[tarifa,'LimI']
    else:
        kwh_nv = (gasto - dac.loc[region,'c_fijo_m'])/dac.loc[region,'c_ener_m']
    
    #summer
    if (tarifa == '1') & (gasto <= (cfe.loc[tarifa,'Ldac']*dac.loc[region,'c_ener_m_sm']+dac.loc[region,'c_fijo_v_m'])*tol_dac):
        kwh_v = kwh_nv
    elif (tarifa == '1') & (gasto > (cfe.loc[tarifa,'Ldac']*dac.loc[region,'c_ener_m_sm']+dac.loc[region,'c_fijo_v_m'])*tol_dac):
        kwh_v = (gasto - dac.loc[region,'c_fijo_v_m'])/dac.loc[region,'c_ener_m_sm']
    else:    
        if gasto <= cfe.loc[tarifa,'LBV_pesos']:
            kwh_v = gasto/cfe.loc[tarifa,'BasicoV']
        elif gasto <= cfe.loc[tarifa,'LIV_pesos']:
            kwh_v = (gasto - cfe.loc[tarifa,'LBV_pesos'])/cfe.loc[tarifa,'IntermedioV'] + cfe.loc[tarifa,'LimBV']
        elif gasto <= cfe.loc[tarifa,'LIVA_pesos']:
            kwh_v = (gasto - cfe.loc[tarifa,'LIV_pesos'])/cfe.loc[tarifa,'IntermedioVA'] + cfe.loc[tarifa,'LimIV']
        #elif gasto <= cfe.loc[tarifa,'LdacV_pesos']*tol_dac:
        elif gasto <= (cfe.loc[tarifa,'Ldac']*dac.loc[region,'c_ener_m_sm']+dac.loc[region,'c_fijo_v_m'])*tol_dac:
            if np.isnan(cfe.loc[tarifa,'LIVA_pesos']):
                kwh_v = (gasto - cfe.loc[tarifa,'LIV_pesos'])/cfe.loc[tarifa,'Excedente'] + cfe.loc[tarifa,'LimIV']
            else:
                kwh_v = (gasto - cfe.loc[tarifa,'LIVA_pesos'])/cfe.loc[tarifa,'Excedente'] + cfe.loc[tarifa,'LimIVA']
        else:
            kwh_v = (gasto - dac.loc[region,'c_fijo_v_m'])/dac.loc[region,'c_ener_m_sm']
            
    if tipo_tar:         
        if (kwh_nv+kwh_v)/2 > cfe.loc[tarifa,'Ldac']:
            return 'DAC'
        else:
            return '1X'
    else:
        return (kwh_nv+kwh_v)/2

In [44]:
# Pesos to energy
def converter(gasto, tipo, tanque = None):
    # gasto: pesos 
    # tipo: lpg, natural, magna, premium, diesel
      
    if tipo == 'lpg':
        kwh_kg = 12.79 # kwh/kg
        mxn_l = 10.34
        mxn_kg = 19.19
        p_gas = 0.6 # kg/l

        if tanque == 1:
            kg_gas = gasto/mxn_l*p_gas
        elif tanque == 2:
            kg_gas = gasto/mxn_kg
            
        kwh_comb = kg_gas*kwh_kg
        
    elif tipo == 'natural':
        kwh_m3 = 8967600*0.000001163 # cal/m3 * kwh/cal
        mxn_gj = 176
        
        kwh_comb = gasto/mxn_gj * 277.778 # kwh/gj
        
    elif tipo == 'magna':
        mxn_magna_l = 17.61
        cal_magna = 43.3   # MJ/kg
        p_magna = 0.720  # kg/l
        
        kwh_comb = gasto/mxn_magna_l * cal_magna * p_magna * 0.2778 # kwh/MJ
        
    elif tipo == 'premium':
        mxn_premium_l = 19.13
        cal_premium = 44.5 # MJ/kg
        p_premium = 0.7057  # kg/l
        
        kwh_comb = gasto/mxn_premium_l * cal_premium * p_premium * 0.2778 # kwh/MJ
        
    elif tipo == 'diesel':
        mxn_diesel_l = 18.76
        cal_diesel = 45 # MJ/kg
        p_diesel = 0.8367  # kg/l
        
        kwh_comb = gasto/mxn_diesel_l * p_diesel * cal_diesel * 0.2778 # kwh/MJ
        
    
    return(kwh_comb)

In [45]:
converter(387,'lpg',2)

257.9327774882751

## Calculation

In [46]:
# Take rows of CFE's costumers
ele = enigh_tar.loc[(enigh_tar['disp_elect']==1)].copy() #& (~ZM_gi['gasto_tri_ele'].isnull())
ele['kwh'] = np.vectorize(mxn_to_kwh)(ele['gasto_tri_ele']/3, ele['Tarifa'], ele['Region']) 

In [47]:
enigh_tar['lpg_kwh'] = np.vectorize(converter)(enigh_tar['gasto_tri_lpg']/3, 'lpg', enigh_tar['tanque_gas']) 
enigh_tar['g_natural_kwh'] = np.vectorize(converter)(enigh_tar['gasto_tri_gas']/3, 'natural') 
enigh_tar['magna_kwh'] = np.vectorize(converter)(enigh_tar['gasto_tri_Magna']/3, 'magna') 
enigh_tar['premium_kwh'] = np.vectorize(converter)(enigh_tar['gasto_tri_Premium']/3, 'premium') 
enigh_tar['diesel_kwh'] = np.vectorize(converter)(enigh_tar['gasto_tri_diesel']/3, 'diesel') 

In [48]:
enigh_tar = enigh_tar.merge(ele[['folioviv','foliohog','kwh']], on=['folioviv','foliohog'], how='left')

In [49]:
fuel_list = ['magna_kwh','premium_kwh','diesel_kwh']
enigh_tar['fuel'] = enigh_tar[fuel_list].sum(axis=1, min_count=1)
enigh_tar['gas'] = enigh_tar[['lpg_kwh','g_natural_kwh']].sum(axis=1, min_count=1)
enigh_tar['total_ener'] = enigh_tar[['fuel','gas','kwh']].sum(axis=1, min_count=1)

In [50]:
enigh_tar.describe()

,folioviv,foliohog,CVE_MUN,estado,publico,CVE_ZM,CVE_ENT,tam_loc,est_socio,est_dis,upm,factor,clase_hog,sexo_jefe,edad_jefe,educa_jefe,tot_integ,hombres,mujeres,mayores,menores,ocupados,ing_cor,gasto_mon,energia,vehiculos,tipo_viv,antiguedad,num_cuarto,disp_elect,focos_inca,focos_ahor,combustible,tenencia,calent_sol,calent_gas,tanque_gas,aire_acond,calefacc,gasto_tri_ele,gasto_tri_gas,gasto_tri_lpg,gasto_tri_oil,gasto_tri_diesel,gasto_tri_coal,gasto_tri_wood,gasto_tri_heat,gasto_tri_Magna,gasto_tri_Premium,gasto_tri_Die-Gas,gasto_tri_total_gas,gasto_tri_total_gasolina,lpg_kwh,g_natural_kwh,magna_kwh,premium_kwh,diesel_kwh,kwh,fuel,gas,total_ener
count,3.110000e+04,31100.000000,31100.000000,31100.000000,31100.000000,31100.000000,31100.000000,31100.000000,31100.000000,31100.000000,31100.000000,31100.000000,31100.000000,31100.000000,31100.000000,31100.000000,31100.000000,31100.000000,31100.000000,31100.000000,31100.000000,31100.000000,3.110000e+04,3.110000e+04,31100.000000,31100.000000,31070.000000,25076.00000,31100.000000,31100.000000,31043.000000,31043.000000,31100.000000,31100.000000,31100.000000,31100.000000,31100.000000,31100.000000,31100.000000,28303.000000,3686.000000,16574.000000,18.000000,10.000000,864.000000,449.000000,129.000000,13909.000000,1393.000000,69.000000,21170.000000,29174.000000,16574.000000,3686.000000,13909.000000,1393.000000,10.000000,28239.000000,15096.000000,20102.000000,30078.000000
mean,1.532120e+09,1.019035,15314.775659,15.283859,1693.304214,14.836921,15.283859,1.345145,2.591961,238.862733,4082.892958,644.437395,2.154630,1.309936,49.518585,6.386399,3.507621,1.698810,1.808810,2.838907,0.668714,1.680096,5.724480e+04,3.585690e+04,1571.029562,0.756109,1.142356,23.30910,3.976270,1.019100,1.279161,6.452985,3.119357,3.073441,1.936849,1.497878,1.870579,1.749839,1.955177,835.292990,929.439699,1249.081883,676.347222,1254.372000,249.118333,550.656704,1101.731473,5098.900246,6501.117753,3133.138841,1139.735374,2741.367675,283.827503,488.973297,835.888863,988.243980,233.123893,214.164352,861.508565,323.674888,849.778344
std,9.164128e+08,0.162651,9177.380002,9.167614,2448.514643,9.203392,9.167614,0.675375,0.751045,154.753525,2386.560107,624.722751,0.647871,0.462474,15.517420,2.529301,1.769338,1.132721,1.169942,1.370390,0.948525,1.058990,6.630378e+04,3.428771e+04,1532.746407,0.921552,0.533091,15.31079,1.555582,0.250062,2.594685,5.662763,0.573231,1.230609,0.243238,0.500004,0.335671,0.433112,0.206919,1152.479338,728.367251,834.671480,873.242448,1821.187653,337.073538,882.672515,703.764174,4294.313899,5404.315484,3218.587592,852.759146,4193.036139,197.629444,383.190148,703.988898,821.517537,338.466065,189.927650,731.225291,258.312774,849.919699
min,1.000136e+08,1.000000,1001.000000,1.000000,0.000000,1.010000,1.000000,1.000000,1.000000,1.000000,1.000000,42.000000,1.000000,1.000000,14.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,1.000000,0.00000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,12.500000,25.000000,104.510000,39.000000,52.250000,0.000000,29.030000,42.000000,58.060000,174.190000,60.960000,0.000000,0.000000,24.660240,13.152367,9.518074,26.478865,9.710615,4.529576,9.518074,13.152367,6.522590
25%,8.016208e+08,1.000000,8019.000000,8.000000,0.000000,7.020000,8.000000,1.000000,2.000000,116.000000,2131.000000,259.000000,2.000000,1.000000,38.000000,4.000000,2.000000,1.000000,1.000000,2.000000,0.000000,1.000000,2.722017e+04,1.753493e+04,580.640000,0.000000,1.000000,12.00000,3.000000,1.000000,0.000000,3.000000,3.000000,2.000000,2.000000,1.000000,2.000000,1.000000,2.000000,300.000000,540.000000,600.000000,106.207500,58.785000,87.090000,150.000000,580.640000,2322.580000,2903.220000,870.960000,580.640000,0.000000,141.896352,284.091136,380.752449,441.322523,10.925139,106.118345,380.752449,161.246865,284.904524
50%,1.507773e+09,1.000000,15033.000000,15.000000,822.830000,14.010000,15.000000,1.000000,2.0000

In [51]:
enigh_tar.to_pickle(pkls_path + '/total_energy.pkl')

## Municipalities consumption map

In [52]:
total_mun = ele[['CVE_ZM', 'CVE_MUN', 'kwh', 'gas', 'fuel']].groupby(['CVE_ZM','CVE_MUN'], as_index = False).sum()

KeyError: "['gas', 'fuel'] not in index"

In [ ]:
shp_file = 'D:/Tesis/Datos/shapefiles/MUN/MUN.shp'
mun_shp = gpd.read_file(shp_file)

In [ ]:
mun_shp = mun_shp.merge(total_mun, on=['CVE_MUN'], how='left' )
mun_shp = mun_shp.loc[~mun_shp['CVE_ZM'].isnull()]

In [ ]:
mun_shp.loc[mun_shp['CVE_ZM']==9.01].to_file("shapes/cdmx_consumption.shp")
mun_shp.loc[mun_shp['CVE_ZM']==19.01].to_file("shapes/mty_consumption.shp")
mun_shp.loc[mun_shp['CVE_ZM']==14.01].to_file("shapes/gdl_consumption.shp")